# 1.1 Парсинг данных

### 1.1.* Парсинг статаей с веб-ресурсов 

Подключение библиотек

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [3]:
uri = "https://habr.com/ru/all/"
page = requests.get(uri) 
print(page.status_code)

200


Создание словаря
- name - назваение статьи 
- name_company - кампания-автор
- date - дата пуьликации
- note - текст статьи

In [ ]:
article = {"name":[],"name_company":[], "date":[], "note":[]}

In [ ]:
# пребор страниц Habr от первойдо третей 
for i in range(1, 3):
    
    # переход на i-ю страницу 
    uri_page = "{0}page{1}/".format(uri, i)
    page = requests.get(uri_page)
    print("Страница: ", i, page)
    soup = bs(page.text,  'html.parser')
    snippetsoup_name = soup.find_all('h2', class_='tm-title tm-title_h2')
    
    for i in snippetsoup_name:
        
        # переход на i-ю статью страници
        uri_article = "https://habr.com" + str(i.a.get("href"))
        page_article = requests.get(uri_article)
        
        soup_article = bs(page_article.text,  'html.parser')
        descriptions_article = soup_article.find_all('a', class_='tm-article-snippet__hubs-item-link')
        
        for description in descriptions_article:
            for j in description.span.text.split():
                # проверка на компанию-автора статьи
                if (companies_df['name_company'].eq(j)).any():
                    print("    Мы взяли статью с странице: ", uri_article)
                    h1 = soup_article.find('h1', class_='tm-title tm-title_h1')
                    article["name"].append(h1.span.text)
                    article["name_company"].append(j)
                    date = soup_article.find('span', class_='tm-article-datetime-published')
                    article["date"].append(date.time.get("title"))
                    time = soup_article.find('span', class_='tm-article-reading-time__label')
                    # две реализачии HTML структуры документа
                    try:
                        note = soup_article.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1')
                        article["note"].append(note.div.text.replace("\n", " ").replace("\r", " ").replace("  ", " ").replace("\xa0", " "))
                    except:
                        note = soup_article.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2')
                        article["note"].append(note.div.text.replace("\n", " ").replace("\r", " ").replace("  ", " ").replace("\xa0", " "))

In [ ]:
articles_df = pd.DataFrame(article)
articles_df

Сохранить в сsv формате

In [ ]:
file_name = "Статьи.csv"
articles_df.to_csv(file_name, encoding="utf-8-sig")

Мы получили файл "Статьи.csv" cо статьями с сайта  habr.com - название, компания-автор, дата публикации, текст статиь 

### 1.1.* Парсинг данных c PDF документа(ов)

Установка нужных библиотек

In [ ]:
pip install PyMuPDF

In [ ]:
pip install fitz 

Подключение библиотек

In [ ]:
import glob 
import fitz 
import pandas as pd 

all_documents = glob.glob(r"C:\Users\nudan\Спец курс\Pdf parsing\*.pdf")

In [ ]:
for i in range(len(all_documents)):
    print("{0}: {1}".format(i, all_documents[i]))

In [ ]:
companies_path_doc = all_documents[3]
companies_doc = fitz.open(companies_path_doc)
print("Открытый документ: ", companies_doc)
print("Число страниц: ",companies_doc.page_count)
text = ""
for page_doc in range(len(companies_doc)):
    page = companies_doc.load_page(page_doc)
    text += page.get_text("text")
text = text.split("\n")
for i in text:
    print(i)

In [ ]:
company_columns = {"name_company":[], "description":[], "activity":[], "rating":[]}

In [ ]:
for i in range(0, len(text), 6):
    company_columns["name_company"].append(text[i])
    company_columns["description"].append(text[i+1])
    company_columns["rating"].append(text[i + 4])
    # удаления в начаое строки патерна "Пишет в хабы:" и добавления второй строки 
    text_new = text[i+2][text[i+2].index(":")+1:] + text[i+3]
    
    # удаление неизвестный символов, перед описанием сферы деятельность
    text_new = text_new.split(",")
    text_new[0] = text_new[0] + " "
    for i in range(1, len(text_new)):
        text_new[i] = text_new[i][1:]
    text_new = ''.join(text_new) 
    
    company_columns["activity"].append(text_new) 

In [ ]:
companies_df = pd.DataFrame(company_columns)
companies_df

Соединение статей и описание компании

In [ ]:
df = articles_df.merge(companies_df, on='name_company', how='left')

# 1.2 Формирование структуры набора данных

Подключение библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.head()

#### Описание данных

In [ ]:
df.info()

In [ ]:
# объём данных (строк и столбцов - объектов и признаков)
df.shape

In [ ]:
df.describe()

In [ ]:
df.hist(figsize = (15,10), bins = 50)

In [ ]:
for col in df.columns:
    print(f'Процент пропусков в колонке {col} = {df[col].isna().sum() / df.shape[0] * 100: .2f}')

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.isna())

In [ ]:
# Удалим переменную Cabin (номер каюты)
df.drop(columns = 'Cabin', axis = 1, inplace = True)

In [ ]:
Отчет, то то сделано, те то и то то признаки взяты 

# 1.3 Предварительная обработка текстовых данных  

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
covid2_drop[covid2_drop["continent"].isnull()]["location"].value_counts()

In [6]:
def edit_text(text):
    text = ' '.join([i if len(i) >= 3 else ' ' for i in text.split(" ")])
    text = ''.join([i if i.isalpha() else ' ' for i in text])
    text = text.lower()
    text = text.replace('ё', 'е')
    text = text.replace('й', 'и')   
    text = re.sub('[a-z]','', text, flags=re.I)
    text = re.sub(r'\s+',' ', text, flags=re.I)
    return text

In [ ]:
prep_text3 = [edit_text(text) for text in df['text_article'] ]

In [ ]:
df['prep_text2'] = prep_text3

# 1.4  Отчет